In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
# Calculate the total number of schools
total_schools = len(school_data_complete["School ID"].unique())

# Calculate the total number of students
total_students = len(school_data_complete["Student ID"].unique())

# Calculate the total budget
total_budget = school_data["budget"].sum()

# Calculate the average math score
avg_math_score = school_data_complete["math_score"].mean()

# Calculate the average reading score
avg_reading_score = school_data_complete["reading_score"].mean()

In [3]:
# Create bins to place score values
bins = [0, 69, 100]

# Create labels for these bins
score_labels = ["Fail", "Pass"]

# Place the sliced bin data into new columns inside of the data, for math and reading
school_data_complete["Math Grade Group"] = pd.cut(school_data_complete["math_score"], bins, labels=score_labels)
school_data_complete["Reading Grade Group"] = pd.cut(school_data_complete["reading_score"], bins, labels=score_labels)

In [4]:
# Isolate passing scores for math, reading & all
pass_math = school_data_complete.loc[(
    school_data_complete["Math Grade Group"] == "Pass")]
pass_reading = school_data_complete.loc[(
    school_data_complete["Reading Grade Group"] == "Pass")]
pass_overall = school_data_complete.loc[(
    school_data_complete["Math Grade Group"] == "Pass") & 
    (school_data_complete["Reading Grade Group"] == "Pass")]

# Calculate the percentage of students with a passing math score (70 or greater)
perc_pass_math = (pass_math["Student ID"].count() / total_students) * 100
perc_pass_reading = (pass_reading["Student ID"].count() / total_students) * 100
perc_pass_overall = (pass_overall["Student ID"].count() / total_students) * 100

In [5]:
# Create a dataframe to hold the above results
district_summary_df = pd.DataFrame({
    "Total Schools": [total_schools], "Total Students": [total_students],
    "Total Budget": [total_budget], "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_reading_score], "% Passing Math": [perc_pass_math],
    "% Passing Reading": [perc_pass_reading], "% Overall Passing": [perc_pass_overall]
})
district_summary_df
print(district_summary_df)

   Total Schools  Total Students  Total Budget  Average Math Score  \
0             15           39170      24649428           78.985371   

   Average Reading Score  % Passing Math  % Passing Reading  % Overall Passing  
0               81.87784       74.980853          85.805463          65.172326  


In [6]:
# Optional: give the displayed data cleaner formatting
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${0:,.2f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.2f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.2f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.2f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.2f}".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.2f}".format)

district_summary_df
print(district_summary_df)

   Total Schools Total Students    Total Budget Average Math Score  \
0             15         39,170  $24,649,428.00              78.99   

  Average Reading Score % Passing Math % Passing Reading % Overall Passing  
0                 81.88          74.98             85.81             65.17  


In [7]:
# Using GroupBy in order to separate the data into fields according to different schools
grouped_school_df = school_data_complete.groupby(["school_name"])

# Find School Type
school_type = grouped_school_df["type"].unique()

# Find total students per school
school_student_count = grouped_school_df["Student ID"].count()

# Find Total School Budget
school_budget = grouped_school_df["budget"].mean()

# Find Per Student Budget
per_student_budget = school_budget / school_student_count

# Find Average Math Score
school_avg_math_score = grouped_school_df["math_score"].mean()

# Find Average Reading Score
school_avg_reading_score = grouped_school_df["reading_score"].mean()

In [8]:
# Count how many of pass values for each school and create DataFrame
pass_math_df = pd.DataFrame(grouped_school_df["Math Grade Group"].value_counts())

# Rename the "Math Grade Group" column to "Math Counts"
pass_math_df = pass_math_df.rename(
    columns={"Math Grade Group": "Math Counts"})

# Reset the index of "pass_math_df" so that includes all columns
pass_math_df = pass_math_df.reset_index()

# Isolate passing scores for math
sch_pass_math = pass_math_df.loc[(
    pass_math_df["Math Grade Group"] == "Pass")]

# Merge the "school_student_count" into the DataFrame
sch_pass_math = sch_pass_math.merge(school_student_count, on="school_name")

# Add column for % passed
sch_pass_math["% Passed Math"] = (sch_pass_math["Math Counts"] / sch_pass_math["Student ID"]) * 100

# Condense sch_pass_math df to basic categories
sch_pass_math_df = sch_pass_math.drop(columns=["Math Grade Group", "Math Counts", "Student ID"])

In [9]:
# Count how many of pass values for each school and create DataFrame, for reading
pass_rdg_df = pd.DataFrame(grouped_school_df["Reading Grade Group"].value_counts())

# Rename the "Reading Grade Group" column to "Reading Counts"
pass_rdg_df = pass_rdg_df.rename(
    columns={"Reading Grade Group": "Reading Counts"})

# Reset the index of "pass_rdg_df" so that includes all columns
pass_rdg_df = pass_rdg_df.reset_index()

# Isolate passing scores for reading
sch_pass_rdg = pass_rdg_df.loc[(
    pass_rdg_df["Reading Grade Group"] == "Pass")]

# Merge the "school_student_count" into the DataFrame
sch_pass_rdg = sch_pass_rdg.merge(school_student_count, on="school_name")

# Add column for % passed
sch_pass_rdg["% Passed Rdg"] = (sch_pass_rdg["Reading Counts"] / sch_pass_rdg["Student ID"]) * 100

# Condense sch_pass_rdg df to basic categories
sch_pass_rdg_df = sch_pass_rdg.drop(columns=["Reading Grade Group", "Reading Counts", "Student ID"])

In [10]:
# Create new df for just overall passing students
overall_pass_df = school_data_complete.loc[(school_data_complete["Math Grade Group"] == "Pass") &
                                          (school_data_complete["Reading Grade Group"] == "Pass")]

# Using GroupBy in order to separate the overall pass data into fields according to different schools
grouped_school_pass_df = overall_pass_df.groupby(["school_name"])

# Turn into dataframe
pass_all_df = pd.DataFrame(grouped_school_pass_df["size"].value_counts())

# Rename the "Reading Grade Group" column to "Reading Counts"
pass_all_df = pass_all_df.rename(
    columns={"size": "# Passed"})

# Reset the index of "pass_all_df" so that includes all columns
pass_all_df = pass_all_df.reset_index()

# Add column for % passed
pass_all_df["% Passed Overall"] = (pass_all_df["# Passed"] / pass_all_df["size"]) * 100

# Condense pass_all_df to basic categories
sch_pass_all_df = pass_all_df.drop(columns=["size", "# Passed"])

In [11]:
# Group data by school and school type
grouped_school_type = school_data_complete.groupby(["school_name", "type"])

# Turn into dataframe
grouped_school_type_df = pd.DataFrame(grouped_school_type["School ID"].value_counts())

# Rename the "School ID" column to "Student Counts"
grouped_school_type_df = grouped_school_type_df.rename(
    columns={"School ID": "Student Counts"})

# Reset the index so that includes all columns
grouped_school_type_df = grouped_school_type_df.reset_index()

In [12]:
# Create new DataFrame for school_budget
school_budget_df = pd.DataFrame(school_budget)

# Re-set index for data frame
school_budget_df = school_budget_df.reset_index()

# Merge school_budget_df into grouped_school_type_df
grouped_school_type_df = pd.merge(grouped_school_type_df, school_budget_df, on="school_name")

In [13]:
# Add per student budget to grouped_school_type
# Create new DataFrame for per_student_budget
per_student_budget_df = pd.DataFrame(per_student_budget)

# Re-set index for data frame
per_student_budget_df = per_student_budget_df.reset_index()

# Merge per_student_budget_df into grouped_school_type_df
grouped_school_type_df = pd.merge(grouped_school_type_df, per_student_budget_df, on="school_name")

In [14]:
# Add Average Math Score to grouped_school_type
# Create new DataFrame for school_avg_math_score
school_avg_math_score_df = pd.DataFrame(school_avg_math_score)

# Re-set index for data frame
school_avg_math_score_df = school_avg_math_score_df.reset_index()

# Merge per_student_budget_df into grouped_school_type_df
grouped_school_type_df = pd.merge(grouped_school_type_df, school_avg_math_score_df, on="school_name")

In [15]:
# Add Average Reading Score to grouped_school_type
# Create new DataFrame for school_avg_reading_score
school_avg_reading_score_df = pd.DataFrame(school_avg_reading_score)

# Re-set index for data frame
school_avg_reading_score_df = school_avg_reading_score_df.reset_index()

# Merge per_student_budget_df into grouped_school_type_df
grouped_school_type_df = pd.merge(grouped_school_type_df, school_avg_reading_score_df, on="school_name")

# Add % Passing Math to grouped_school_type
# Merge sch_pass_math_df into grouped_school_type_df
grouped_school_type_df = pd.merge(grouped_school_type_df, sch_pass_math_df, on="school_name")

# Add % Passing Reading to grouped_school_type
# Merge sch_pass_rdg_df into grouped_school_type_df
grouped_school_type_df = pd.merge(grouped_school_type_df, sch_pass_rdg_df, on="school_name")

# Add % Overall Passing to grouped_school_type
# Merge sch_pass_all_df into grouped_school_type_df
grouped_school_type_df = pd.merge(grouped_school_type_df, sch_pass_all_df, on="school_name")

In [16]:
# Remove School Id column
grouped_school_type_df = grouped_school_type_df.drop(columns=["School ID"])

In [17]:
# Rename columns; had to use 0 (not in quotes) for unnamed column
final_school_summary = grouped_school_type_df.rename(columns={"school_name": "School Name", "type": "School Type", 
                                      "Student Counts": "Total Students", "budget": "Total School Budget", 
                                      0: "Per Student Budget", "math_score": "Average Math Score", 
                                      "reading_score": "Average Reading Score", "% Passed Math": "% Passing Math", 
                                      "% Passed Rdg": "% Passing Reading", "% Passed Overall": "% Overall Passing"})

In [18]:
# Make copy of this to be able to use numerical data later before formatting column
final_school_sum_copy = final_school_summary.copy()
print(final_school_sum_copy.head())

            School Name School Type  Total Students  Total School Budget  \
0    Bailey High School    District            4976              3124928   
1   Cabrera High School     Charter            1858              1081356   
2  Figueroa High School    District            2949              1884411   
3      Ford High School    District            2739              1763916   
4   Griffin High School     Charter            1468               917500   

   Per Student Budget  Average Math Score  Average Reading Score  \
0               628.0           77.048432              81.033963   
1               582.0           83.061895              83.975780   
2               639.0           76.711767              81.158020   
3               644.0           77.102592              80.746258   
4               625.0           83.351499              83.816757   

   % Passing Math  % Passing Reading  % Overall Passing  
0       66.680064          81.933280          54.642283  
1       94.133477 

In [19]:
# Format Total School Budget and Per Student Budget for currency
final_school_summary["Total School Budget"] = final_school_summary["Total School Budget"].map("${0:,.2f}".format)
final_school_summary["Per Student Budget"] = final_school_summary["Per Student Budget"].map("${0:,.2f}".format)
print(final_school_summary)

              School Name School Type  Total Students Total School Budget  \
0      Bailey High School    District            4976       $3,124,928.00   
1     Cabrera High School     Charter            1858       $1,081,356.00   
2    Figueroa High School    District            2949       $1,884,411.00   
3        Ford High School    District            2739       $1,763,916.00   
4     Griffin High School     Charter            1468         $917,500.00   
5   Hernandez High School    District            4635       $3,022,020.00   
6      Holden High School     Charter             427         $248,087.00   
7       Huang High School    District            2917       $1,910,635.00   
8     Johnson High School    District            4761       $3,094,650.00   
9        Pena High School     Charter             962         $585,858.00   
10  Rodriguez High School    District            3999       $2,547,363.00   
11    Shelton High School     Charter            1761       $1,056,600.00   

In [20]:
# Sort and display the top five performing schools by % overall passing.
# Sort the DataFrame by the values in the "% Overall Passing" column to find the best
# To sort from highest to lowest, ascending=False must be passed in
top_pass = final_school_summary.sort_values("% Overall Passing", ascending=False)

# Reset the index so that the index is now based on the sorting locations
top_pass = top_pass.reset_index(drop=True)
print(top_pass.head())

           School Name School Type  Total Students Total School Budget  \
0  Cabrera High School     Charter            1858       $1,081,356.00   
1   Thomas High School     Charter            1635       $1,043,130.00   
2  Griffin High School     Charter            1468         $917,500.00   
3   Wilson High School     Charter            2283       $1,319,574.00   
4     Pena High School     Charter             962         $585,858.00   

  Per Student Budget  Average Math Score  Average Reading Score  \
0            $582.00           83.061895              83.975780   
1            $638.00           83.418349              83.848930   
2            $625.00           83.351499              83.816757   
3            $578.00           83.274201              83.989488   
4            $609.00           83.839917              84.044699   

   % Passing Math  % Passing Reading  % Overall Passing  
0       94.133477          97.039828          91.334769  
1       93.272171          97.308869

In [21]:
# Sort and display the five worst-performing schools by % overall passing.
# Sort the DataFrame by the values in the "% Overall Passing" column to find the worst
worst_pass = final_school_summary.sort_values("% Overall Passing")

# Reset the index so that the index is now based on the sorting locations
worst_pass = worst_pass.reset_index(drop=True)
print(worst_pass.head())

             School Name School Type  Total Students Total School Budget  \
0  Rodriguez High School    District            3999       $2,547,363.00   
1   Figueroa High School    District            2949       $1,884,411.00   
2      Huang High School    District            2917       $1,910,635.00   
3  Hernandez High School    District            4635       $3,022,020.00   
4    Johnson High School    District            4761       $3,094,650.00   

  Per Student Budget  Average Math Score  Average Reading Score  \
0            $637.00           76.842711              80.744686   
1            $639.00           76.711767              81.158020   
2            $655.00           76.629414              81.182722   
3            $652.00           77.289752              80.934412   
4            $650.00           77.072464              80.966394   

   % Passing Math  % Passing Reading  % Overall Passing  
0       66.366592          80.220055          52.988247  
1       65.988471       

In [22]:
# Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) 
# at each school.

# Create a pandas series for each grade. Hint: use a conditional statement.
ninth_grade = school_data_complete.loc[(school_data_complete["grade"] == "9th")]
tenth_grade = school_data_complete.loc[(school_data_complete["grade"] == "10th")]
eleventh_grade = school_data_complete.loc[(school_data_complete["grade"] == "11th")]
twelvth_grade = school_data_complete.loc[(school_data_complete["grade"] == "12th")]

# Group each table by school_name
grp_nine = ninth_grade.groupby(["school_name"])
grp_ten = tenth_grade.groupby(["school_name"])
grp_eleven = eleventh_grade.groupby(["school_name"])
grp_twelve = twelvth_grade.groupby(["school_name"])

# Find average math scores
nine_avg_math = grp_nine["math_score"].mean()
ten_avg_math = grp_ten["math_score"].mean()
eleven_avg_math = grp_eleven["math_score"].mean()
twelve_avg_math = grp_twelve["math_score"].mean()

# Create data frames for each grade
nine_avg_math_df = pd.DataFrame(nine_avg_math)
ten_avg_math_df = pd.DataFrame(ten_avg_math)
eleven_avg_math_df = pd.DataFrame(eleven_avg_math)
twelve_avg_math_df = pd.DataFrame(twelve_avg_math)

# Re-set index for data frame
nine_avg_math_df = nine_avg_math_df.reset_index()
ten_avg_math_df = ten_avg_math_df.reset_index()
eleven_avg_math_df = eleven_avg_math_df.reset_index()
twelve_avg_math_df = twelve_avg_math_df.reset_index()

# Merge other grades into ninth grade df and rename columns
avg_math_df = pd.merge(nine_avg_math_df, ten_avg_math_df, on="school_name")
avg_math_df = avg_math_df.rename(columns={"math_score_x": "9th"})
avg_math_df = avg_math_df.rename(columns={"math_score_y": "10th"})
avg_math_df = pd.merge(avg_math_df, eleven_avg_math_df, on="school_name")
avg_math_df = avg_math_df.rename(columns={"math_score": "11th"})
avg_math_df = pd.merge(avg_math_df, twelve_avg_math_df, on="school_name")
avg_math_df = avg_math_df.rename(columns={"math_score": "12th"})

print(avg_math_df)

              school_name        9th       10th       11th       12th
0      Bailey High School  77.083676  76.996772  77.515588  76.492218
1     Cabrera High School  83.094697  83.154506  82.765560  83.277487
2    Figueroa High School  76.403037  76.539974  76.884344  77.151369
3        Ford High School  77.361345  77.672316  76.918058  76.179963
4     Griffin High School  82.044010  84.229064  83.842105  83.356164
5   Hernandez High School  77.438495  77.337408  77.136029  77.186567
6      Holden High School  83.787402  83.429825  85.000000  82.855422
7       Huang High School  77.027251  75.908735  76.446602  77.225641
8     Johnson High School  77.187857  76.691117  77.491653  76.863248
9        Pena High School  83.625455  83.372000  84.328125  84.121547
10  Rodriguez High School  76.859966  76.612500  76.395626  77.690748
11    Shelton High School  83.420755  82.917411  83.383495  83.778976
12     Thomas High School  83.590022  83.087886  83.498795  83.497041
13     Wilson High S

In [23]:
# Perform the same operations as above for reading scores
# Find average reading scores
nine_avg_rdg = grp_nine["reading_score"].mean()
ten_avg_rdg = grp_ten["reading_score"].mean()
eleven_avg_rdg = grp_eleven["reading_score"].mean()
twelve_avg_rdg = grp_twelve["reading_score"].mean()

# Create data frames for each grade
nine_avg_rdg_df = pd.DataFrame(nine_avg_rdg)
ten_avg_rdg_df = pd.DataFrame(ten_avg_rdg)
eleven_avg_rdg_df = pd.DataFrame(eleven_avg_rdg)
twelve_avg_rdg_df = pd.DataFrame(twelve_avg_rdg)

# Re-set index for data frame
nine_avg_rdg_df = nine_avg_rdg_df.reset_index()
ten_avg_rdg_df = ten_avg_rdg_df.reset_index()
eleven_avg_rdg_df = eleven_avg_rdg_df.reset_index()
twelve_avg_rdg_df = twelve_avg_rdg_df.reset_index()

# Merge other grades into ninth grade df and rename columns
avg_rdg_df = pd.merge(nine_avg_rdg_df, ten_avg_rdg_df, on="school_name")
avg_rdg_df = avg_rdg_df.rename(columns={"reading_score_x": "9th"})
avg_rdg_df = avg_rdg_df.rename(columns={"reading_score_y": "10th"})
avg_rdg_df = pd.merge(avg_rdg_df, eleven_avg_rdg_df, on="school_name")
avg_rdg_df = avg_rdg_df.rename(columns={"reading_score": "11th"})
avg_rdg_df = pd.merge(avg_rdg_df, twelve_avg_rdg_df, on="school_name")
avg_rdg_df = avg_rdg_df.rename(columns={"reading_score": "12th"})

print(avg_rdg_df)

              school_name        9th       10th       11th       12th
0      Bailey High School  81.303155  80.907183  80.945643  80.912451
1     Cabrera High School  83.676136  84.253219  83.788382  84.287958
2    Figueroa High School  81.198598  81.408912  80.640339  81.384863
3        Ford High School  80.632653  81.262712  80.403642  80.662338
4     Griffin High School  83.369193  83.706897  84.288089  84.013699
5   Hernandez High School  80.866860  80.660147  81.396140  80.857143
6      Holden High School  83.677165  83.324561  83.815534  84.698795
7       Huang High School  81.290284  81.512386  81.417476  80.305983
8     Johnson High School  81.260714  80.773431  80.616027  81.227564
9        Pena High School  83.807273  83.612000  84.335938  84.591160
10  Rodriguez High School  80.993127  80.629808  80.864811  80.376426
11    Shelton High School  84.122642  83.441964  84.373786  82.781671
12     Thomas High School  83.728850  84.254157  83.585542  83.831361
13     Wilson High S

In [24]:
# Create a table that breaks down school performances based on average Spending Ranges (Per Student).

# Use 4 reasonable bins to group school spending; add to df 
# Create bins to place per student spending values
spending_bins = [0, 583, 629, 644, 675]

# Create labels for these bins
spending_labels = ["<$584", "$585-629", "$630-644", "$645-675"]

# Place the sliced bin data into new column inside of the data
# Use copy of original school summary where columns are still integers
final_school_sum_copy["Spending Ranges (Per Student)"] = pd.cut(final_school_sum_copy["Per Student Budget"], 
                                                                spending_bins, labels=spending_labels)

# Group table by per student spending ranges
grp_sch_sum_spd = final_school_sum_copy.groupby(["Spending Ranges (Per Student)"])

# Create data frame for new grouping
grp_sch_sum_spd_df = pd.DataFrame(grp_sch_sum_spd["School Name"].count())

# Re-set index for data frame
grp_sch_sum_spd_df = grp_sch_sum_spd_df.reset_index()

In [25]:
# Include additional columns for Avg Math, Avg Rdg, % Pass Math, % Pass Rdg, % Overall Pass
# Find average values
avg_math_spd = grp_sch_sum_spd["Average Math Score"].mean()
avg_rdg_spd = grp_sch_sum_spd["Average Reading Score"].mean()
pass_math_spd = grp_sch_sum_spd["% Passing Math"].mean()
pass_rdg_spd = grp_sch_sum_spd["% Passing Reading"].mean()
pass_all_spd = grp_sch_sum_spd["% Overall Passing"].mean()

# Create data frames for values
avg_math_spd_df = pd.DataFrame(avg_math_spd)
avg_rdg_spd_df = pd.DataFrame(avg_rdg_spd)
pass_math_spd_df = pd.DataFrame(pass_math_spd)
pass_rdg_spd_df = pd.DataFrame(pass_rdg_spd)
pass_all_spd_df = pd.DataFrame(pass_all_spd)

# Re-set index for value data frames
avg_math_spd_df = avg_math_spd_df.reset_index()
avg_rdg_spd_df = avg_rdg_spd_df.reset_index()
pass_math_spd_df = pass_math_spd_df.reset_index()
pass_rdg_spd_df = pass_rdg_spd_df.reset_index()
pass_all_spd_df = pass_all_spd_df.reset_index()

# Merge data frames together
spd_df = pd.merge(avg_math_spd_df, avg_rdg_spd_df, on="Spending Ranges (Per Student)")
spd_df = pd.merge(spd_df, pass_math_spd_df, on="Spending Ranges (Per Student)")
spd_df = pd.merge(spd_df, pass_rdg_spd_df, on="Spending Ranges (Per Student)")
spd_df = pd.merge(spd_df, pass_all_spd_df, on="Spending Ranges (Per Student)")

# Re-format table to round to 2 decimals
spd_df["Average Math Score"] = spd_df["Average Math Score"].map("{:.2f}".format)
spd_df["Average Reading Score"] = spd_df["Average Reading Score"].map("{:.2f}".format)
spd_df["% Passing Math"] = spd_df["% Passing Math"].map("{:.2f}".format)
spd_df["% Passing Reading"] = spd_df["% Passing Reading"].map("{:.2f}".format)
spd_df["% Overall Passing"] = spd_df["% Overall Passing"].map("{:.2f}".format)

print(spd_df)

  Spending Ranges (Per Student) Average Math Score Average Reading Score  \
0                         <$584              83.46                 83.93   
1                      $585-629              81.90                 83.16   
2                      $630-644              78.52                 81.62   
3                      $645-675              77.00                 81.03   

  % Passing Math % Passing Reading % Overall Passing  
0          93.46             96.61             90.37  
1          87.13             92.72             81.42  
2          73.48             84.39             62.86  
3          66.16             81.13             53.53  


In [26]:
# Perform the same operations as above, based on school size.
# Create bins to Total Students values
size_bins = [0, 999, 1999, 5000]

# Create labels for these bins
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Place the sliced bin data into new column inside of the data
# Use copy of original school summary where columns are still integers
final_school_sum_copy["School Size"] = pd.cut(final_school_sum_copy["Total Students"], 
                                              size_bins, labels=size_labels)

print(final_school_sum_copy.head())

            School Name School Type  Total Students  Total School Budget  \
0    Bailey High School    District            4976              3124928   
1   Cabrera High School     Charter            1858              1081356   
2  Figueroa High School    District            2949              1884411   
3      Ford High School    District            2739              1763916   
4   Griffin High School     Charter            1468               917500   

   Per Student Budget  Average Math Score  Average Reading Score  \
0               628.0           77.048432              81.033963   
1               582.0           83.061895              83.975780   
2               639.0           76.711767              81.158020   
3               644.0           77.102592              80.746258   
4               625.0           83.351499              83.816757   

   % Passing Math  % Passing Reading  % Overall Passing  \
0       66.680064          81.933280          54.642283   
1       94.13347

In [27]:
# Group table by school size
grp_size = final_school_sum_copy.groupby(["School Size"])

# Create data frame for new grouping
grp_size_df = pd.DataFrame(grp_size["School Name"].count())

# Re-set index for data frame
grp_size_df = grp_size_df.reset_index()

# Include additional columns for Avg Math, Avg Rdg, % Pass Math, % Pass Rdg, % Overall Pass
# Find average values
avg_math_size = grp_size["Average Math Score"].mean()
avg_rdg_size = grp_size["Average Reading Score"].mean()
pass_math_size = grp_size["% Passing Math"].mean()
pass_rdg_size = grp_size["% Passing Reading"].mean()
pass_all_size = grp_size["% Overall Passing"].mean()

# Create data frames for values
avg_math_size_df = pd.DataFrame(avg_math_size)
avg_rdg_size_df = pd.DataFrame(avg_rdg_size)
pass_math_size_df = pd.DataFrame(pass_math_size)
pass_rdg_size_df = pd.DataFrame(pass_rdg_size)
pass_all_size_df = pd.DataFrame(pass_all_size)

# Re-set index for value data frames
avg_math_size_df = avg_math_size_df.reset_index()
avg_rdg_size_df = avg_rdg_size_df.reset_index()
pass_math_size_df = pass_math_size_df.reset_index()
pass_rdg_size_df = pass_rdg_size_df.reset_index()
pass_all_size_df = pass_all_size_df.reset_index()

# Merge data frames together
size_df = pd.merge(avg_math_size_df, avg_rdg_size_df, on="School Size")
size_df = pd.merge(size_df, pass_math_size_df, on="School Size")
size_df = pd.merge(size_df, pass_rdg_size_df, on="School Size")
size_df = pd.merge(size_df, pass_all_size_df, on="School Size")

# Re-format table to round to 2 decimals
size_df["Average Math Score"] = size_df["Average Math Score"].map("{:.2f}".format)
size_df["Average Reading Score"] = size_df["Average Reading Score"].map("{:.2f}".format)
size_df["% Passing Math"] = size_df["% Passing Math"].map("{:.2f}".format)
size_df["% Passing Reading"] = size_df["% Passing Reading"].map("{:.2f}".format)
size_df["% Overall Passing"] = size_df["% Overall Passing"].map("{:.2f}".format)

print(size_df)

          School Size Average Math Score Average Reading Score % Passing Math  \
0       Small (<1000)              83.82                 83.93          93.55   
1  Medium (1000-2000)              83.37                 83.86          93.60   
2   Large (2000-5000)              77.75                 81.34          69.96   

  % Passing Reading % Overall Passing  
0             96.10             89.88  
1             96.79             90.62  
2             82.77             58.29  


In [28]:
# Group table by school type
grp_type = final_school_sum_copy.groupby(["School Type"])

# Create data frame for new grouping
grp_type_df = pd.DataFrame(grp_type["School Name"].count())

# Re-set index for data frame
grp_type_df = grp_type_df.reset_index()

# Include additional columns for Avg Math, Avg Rdg, % Pass Math, % Pass Rdg, % Overall Pass
# Find average values
avg_math_type = grp_type["Average Math Score"].mean()
avg_rdg_type = grp_type["Average Reading Score"].mean()
pass_math_type = grp_type["% Passing Math"].mean()
pass_rdg_type = grp_type["% Passing Reading"].mean()
pass_all_type = grp_type["% Overall Passing"].mean()

# Create data frames for values
avg_math_type_df = pd.DataFrame(avg_math_type)
avg_rdg_type_df = pd.DataFrame(avg_rdg_type)
pass_math_type_df = pd.DataFrame(pass_math_type)
pass_rdg_type_df = pd.DataFrame(pass_rdg_type)
pass_all_type_df = pd.DataFrame(pass_all_type)

# Re-set index for value data frames
avg_math_type_df = avg_math_type_df.reset_index()
avg_rdg_type_df = avg_rdg_type_df.reset_index()
pass_math_type_df = pass_math_type_df.reset_index()
pass_rdg_type_df = pass_rdg_type_df.reset_index()
pass_all_type_df = pass_all_type_df.reset_index()

# Merge data frames together
type_df = pd.merge(avg_math_type_df, avg_rdg_type_df, on="School Type")
type_df = pd.merge(type_df, pass_math_type_df, on="School Type")
type_df = pd.merge(type_df, pass_rdg_type_df, on="School Type")
type_df = pd.merge(type_df, pass_all_type_df, on="School Type")

# Re-format table to round to 2 decimals
type_df["Average Math Score"] = type_df["Average Math Score"].map("{:.2f}".format)
type_df["Average Reading Score"] = type_df["Average Reading Score"].map("{:.2f}".format)
type_df["% Passing Math"] = type_df["% Passing Math"].map("{:.2f}".format)
type_df["% Passing Reading"] = type_df["% Passing Reading"].map("{:.2f}".format)
type_df["% Overall Passing"] = type_df["% Overall Passing"].map("{:.2f}".format)

print(type_df)

  School Type Average Math Score Average Reading Score % Passing Math  \
0     Charter              83.47                 83.90          93.62   
1    District              76.96                 80.97          66.55   

  % Passing Reading % Overall Passing  
0             96.59             90.43  
1             80.80             53.67  
